In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [139]:
# R  W  11111111
# R  P  11111111
# G  W  11111111
# G  P  11111111
# H  W  11111111
# H  P  11111111

# category = 'R'
# category = 'G'
category = 'H'

# bet_type = 'W'
bet_type = 'P'

In [140]:
engine = create_engine('sqlite:///../../data/race.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

5947 races loaded!


In [141]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

1482 H races!


In [142]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head(3)

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
36,37,CHARLTON,VIC,M,H,2017-10-01,1,DNR LOGISTICS 3YO PACE MS,2017-10-01 02:14:00,Paying,2100,"[[1], [5], [3], [4]]",6,"[{""runnerName"": ""TORQUE TOTHE STARZ"", ""runnerN...",3.3,15.6,36.1,57.3,"[[1], [5], [3], [4]]","[{'runnerName': 'TORQUE TOTHE STARZ', 'runnerN..."
37,38,CHARLTON,VIC,M,H,2017-10-01,2,AGISTMENT KILMORE PACE MS,2017-10-01 02:48:00,Paying,1609,"[[1], [2], [8], [7]]",8,"[{""runnerName"": ""SUBTLE EMBRACE"", ""runnerNumbe...",4.3,10.5,33.8,186.1,"[[1], [2], [8], [7]]","[{'runnerName': 'SUBTLE EMBRACE', 'runnerNumbe..."
38,39,CHARLTON,VIC,M,H,2017-10-01,3,VALE ROD GRANT TROT MS,2017-10-01 03:18:00,Paying,2100,"[[5], [1], [6], [2]]",10,"[{""runnerName"": ""SHESASUNDON"", ""runnerNumber"":...",6.5,14.0,91.7,249.9,"[[5], [1], [6], [2]]","[{'runnerName': 'SHESASUNDON', 'runnerNumber':..."


In [143]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
    data_all = data_all.append(race['runners'])    
data_all.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
5,0.054627,0.030660,0.008036,0.007850,0,0,4,"{'returnWin': 41, 'returnWinOpen': 41, 'return...",True,7,...,6.0,0.051898,MITCHELL MILLER,JUST A LEGEND,4,N C DYMOCK,41.00,0.024390,6.0,0.020283
6,0.081704,0.045858,0.006126,0.005985,0,0,0,"{'returnWin': 34, 'returnWinOpen': 26, 'return...",True,7,...,5.0,0.061124,JACK JUSTINS,SIMONDOU SU,5,W JUSTINS,34.00,0.029412,5.0,0.024459
7,0.837974,0.470323,0.792233,0.773943,0,0,1,"{'returnWin': 1.22, 'returnWinOpen': 1.28, 're...",True,7,...,1.0,0.378522,DYLAN EGERTON-GREE,CHIAROSCURO,6,M A LINDAU,1.22,0.819672,1.0,0.681648


In [144]:
# drop scratched
data = data_all.dropna(subset=['win_odds', 'place_odds'])
data = data[(data.win_odds > 0)]
data = data[(data.place_odds > 0)]
data.describe()

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,num_runners,place_odds,place_perc,place_rank,place_scaled,runnerNumber,win_odds,win_perc,win_rank,win_scaled
count,1.420400e+04,1.420400e+04,14204.000000,14204.000000,14204.000000,14204.0,14204.000000,14204.000000,14204.000000,14204.000000,14204.000000,14204.000000,14204.000000,14204.000000,14204.000000,14204.000000,14204.000000
mean,3.006442e-01,1.043368e-01,0.104482,0.104337,0.787032,0.0,0.998029,10.319910,4.585472,0.372427,5.485286,0.104337,5.847156,22.790608,0.130347,5.483033,0.104337
std,2.129554e-01,7.949669e-02,0.141952,0.140169,3.706575,0.0,1.396931,2.988399,26.886431,0.224486,3.304235,0.066524,3.523742,25.298709,0.147079,3.296566,0.117682
min,5.851407e-29,4.824037e-29,0.000000,0.000000,0.000000,0.0,0.000000,4.000000,1.010000,0.000444,1.000000,0.000174,1.000000,1.010000,0.006211,1.000000,0.004005
25%,1.250023e-01,4.197629e-02,0.018346,0.018369,0.000000,0.0,0.000000,8.000000,1.880000,0.190476,3.000000,0.051965,3.000000,5.500000,0.032258,3.000000,0.025858
50%,2.420047e-01,8.099468e-02,0.049272,0.048789,0.000000,0.0,0.000000,10.000000,3.100000,0.322581,5.000000,0.087752,5.000000,14.000000,0.071429,5.000000,0.058692
75%,4.492580e-01,1.565727e-01,0.122293,0.122653,0.000000,0.0,2.000000,12.000000,5.250000,0.531915,8.000000,0.148268,8.000000,31.000000,0.181818,8.000000,0.139587
max,1.000000e+00,7.887297e-01,1.000000,1.000000,80.000000,0.0,4.000000,20.000000,2250.000000,0.990099,20.000000,0.423158,20.000000,161.000000,0.990099,20.000000,0.834143


In [145]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    14204.000000
mean         0.295410
std          0.456243
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
dtype: float64

In [146]:
xp = data['win_perc']
xs = data['win_scaled']
xr = data['win_rank']
xn = data['num_runners']
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()

,win_perc,win_scaled,win_rank,num_runners
count,14204.000000,14204.000000,14204.000000,14204.000000
mean,0.130347,0.104337,5.483033,10.319910
std,0.147079,0.117682,3.296566,2.988399
min,0.006211,0.004005,1.000000,4.000000
25%,0.032258,0.025858,3.000000,8.000000
50%,0.071429,0.058692,5.000000,10.000000
75%,0.181818,0.139587,8.000000,12.000000
max,0.990099,0.834143,20.000000,20.000000


In [147]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 500
print('epochs = {}'.format(epochs))

layer_1 = 30
layer_2 = 30

tag = '{}{}x{}{}'.format(category, layer_1, layer_2, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/each_way/v1/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
    print('model loaded')
except OSError:
    model = Sequential()
    model.add(Dense(layer_1, activation='relu', input_dim=n))
    model.add(Dense(layer_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print('model created')

input dimension = 4
epochs = 500
tag = H30x30P
model loaded


In [148]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/each_way/v1/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)